In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import datasurfer as ds
from datasurfer.lib_llm.utility import agent_tool
from datasurfer.lib_llm.tool import read_file, write_file, web_search, get_current_datetime, list_directory_contents, \
                                     parse_rss_feed
from IPython.display import display, Markdown
from pydantic import BaseModel, Field, ConfigDict

In [19]:
agent = ds.Agent('DAA', config=r"C:\20_Develop\94_AI\70_Endpoints\openrouter.env", 
                 model='nvidia/nemotron-3-nano-30b-a3b:free', 
                 mcp_servers=['http://localhost:8000/mcp'])
# await agent.resp('What time is it now?')
await agent.mcp_resources

[{'name': 'get_greeting',
  'title': None,
  'uri': AnyUrl('resource://greeting'),
  'description': 'Provides a simple greeting message.',
  'mimeType': 'text/plain',
  'size': None,
  'icons': None,
  'annotations': None,
  'meta': {'_fastmcp': {'tags': []}},
  'type': 'resource'},
 {'name': 'get_config',
  'title': None,
  'uri': AnyUrl('data://config'),
  'description': 'Provides application configuration as JSON.',
  'mimeType': 'text/plain',
  'size': None,
  'icons': None,
  'annotations': None,
  'meta': {'_fastmcp': {'tags': []}},
  'type': 'resource'}]

In [3]:
read_file

{'name': 'read_file',
 'description': 'Reads the content of a file.    ',
 'type': 'function',
 'parameters': {'properties': {'file_path': {'description': 'Path to the file',
    'title': 'File Path',
    'type': 'string'},
   'start': {'default': 0,
    'description': 'Starting line to read from',
    'title': 'Start',
    'type': 'integer'},
   'end': {'default': None,
    'description': 'Ending line to read to',
    'title': 'End',
    'type': 'integer'}},
  'required': ['file_path'],
  'type': 'object'}}



https://openai.github.io/openai-agents-python/

https://google.github.io/adk-docs/


In [4]:
class Response(BaseModel):
    model_config = ConfigDict(extra='forbid')
    answer: str = Field(..., description="The answer to the user's question.")

In [4]:
@agent_tool
def get_current_weather(location: str, unit: str = "fahrenheit") -> str:
    """
    Get the current weather and return a summary.
    location: The city and state, e.g. San Francisco, CA
    unit: The unit of temperature. "celsius" or "fahrenheit"
    return: A string describing the current weather   
    """
    return f"It is currently sunny in {location} and 75 degrees {unit}."

@agent_tool
def get_tomorrows_weather(location: str, unit: str = "fahrenheit") -> str:
    """
    Get the weather for tomorrow and return a summary.
    param location: The city and state, e.g. San Francisco, CA
    param unit: The unit of temperature. "celsius" or "fahrenheit"
    return: A string describing the weather for tomorrow
    """
    return f"Tomorrow it will be rainy in {location} and 60 degrees {unit}."

In [5]:
tools = [get_current_weather, get_tomorrows_weather, read_file, write_file, list_directory_contents, parse_rss_feed]

In [6]:
mcp_servers = {'url': 'http://localhost:8000/mcp', 'tools': ['get_current_time']}

In [8]:
agent = ds.Agent('AAA', model='gpt-5-mini', tools=tools, mcp_servers=mcp_servers)
len(await agent.get_available_tools()), len(agent.skills)

[2026-01-21 16:24:59][ERROR][Failed to connect MCP Sevice 'http://localhost:8000/mcp', error: Client failed to connect: All connection attempts failed]


(6, 7)

In [9]:
agent = ds.Agent('AA', skills='null', tools=[get_current_datetime])
Markdown(agent.respond('What is the current date and time?'))

Assistant:

    Function Call:
        Name: get_current_datetime
        Arguments: {}
        Call ID: call_yg6UPyfNXvrKHARBTUxWK1HA


--------------------------------------------------------------------------------------------------------

Assistant:

    The current date and time is January 21, 2026, 16:25:02.

--------------------------------------------------------------------------------------------------------



The current date and time is January 21, 2026, 16:25:02.

In [10]:
agent.history.to_dataframe()

,msg_id,name,role,content,context_level,function_name,call_id,usage,call_output,session_id
0,0,User,user,What is the current date and time?,0,NaN,NaN,NaN,NaN,0
1,1,Assistant,None,NaN,0,get_current_datetime,call_yg6UPyfNXvrKHARBTUxWK1HA,"{'input_tokens': 45, 'input_tokens_details': {...",NaN,0
2,2,System,None,NaN,0,NaN,call_yg6UPyfNXvrKHARBTUxWK1HA,NaN,"""2026-01-21 16:25:02""",0
3,3,Assistant,assistant,"The current date and time is January 21, 2026,...",0,NaN,NaN,"{'input_tokens': 77, 'input_tokens_details': {...",NaN,0


In [11]:
agent = ds.Agent('AAA', model='gpt-5-mini', tools=tools, mcp_servers=mcp_servers, 
                 skills=r'C:\20_Develop\20_Resource\10_Github\skills\awesome-claude-skills')
print(agent.skills.to_text())

{
    "name": "artifacts-builder",
    "description": "Suite of tools for creating elaborate, multi-component claude.ai HTML artifacts using modern frontend web technologies (React, Tailwind CSS, shadcn/ui). Use for complex artifacts requiring state management, routing, or shadcn/ui components - not for simple single-file HTML/JSX artifacts.",
    "license": "Complete terms in LICENSE.txt"
}

{
    "name": "brand-guidelines",
    "description": "Applies Anthropic's official brand colors and typography to any sort of artifact that may benefit from having Anthropic's look-and-feel. Use it when brand colors or style guidelines, visual formatting, or company design standards apply.",
    "license": "Complete terms in LICENSE.txt"
}

{
    "name": "canvas-design",
    "description": "Create beautiful visual art in .png and .pdf documents using design philosophy. You should use this skill when the user asks to create a poster, piece of art, design, or other static piece. Create original visu

In [7]:
agent = ds.Agent('AAA', model='gpt-5-mini', tools=tools, mcp_servers=mcp_servers)
print(agent.skills)
Markdown(await agent.respond('hi, what are your skills provided by user?', compression=True))

Agent Skills: ['docx', 'knowledge-graph', 'parse-rss', 'planning-with-files', 'theme-factory', 'video-downloader', 'web-search']


[2026-01-23 13:51:02][ERROR][Failed to connect MCP Sevice 'http://localhost:8000/mcp', error: Client failed to connect: All connection attempts failed]


Assistant:

    Function Call:
        Name: read_skill_instruction
        Arguments: {"skill_name":"docx"}
        Call ID: call_NwfpQHwtYWR6e0Dcir7aJhYD


    Function Call:
        Name: read_skill_instruction
        Arguments: {"skill_name":"knowledge-graph"}
        Call ID: call_FOmMOGpLxUpSUB3gIGjwnND0


    Function Call:
        Name: read_skill_instruction
        Arguments: {"skill_name":"parse-rss"}
        Call ID: call_ehppp6IewGnjhKYF28sOvxe1


    Function Call:
        Name: read_skill_instruction
        Arguments: {"skill_name":"planning-with-files"}
        Call ID: call_mGe8pm7kY48y7OV4mBRAAWoz


    Function Call:
        Name: read_skill_instruction
        Arguments: {"skill_name":"theme-factory"}
        Call ID: call_KLKtAT2xcxL87R2KRRHvbScw


    Function Call:
        Name: read_skill_instruction
        Arguments: {"skill_name":"youtube-downloader"}
        Call ID: call_hQC8GkMgUT2P3d69gsPJAiD5


    Function Call:
        Name: read_skill_instruction
  

Hi — here are the user-provided skills I see (from the instructions you gave) with short descriptions:

- docx — Comprehensive .docx document creation, editing, and analysis (supports tracked changes, comments, formatting preservation, text extraction, packing/unpacking, and workflows for creating or redlining Word files).  
- knowledge-graph — Guides extracting entities/relationships and returning a knowledge-graph JSON (rules for node IDs, labels, relationship naming, coreference, and strict JSON output).  
- parse-rss — Tools and guidance to fetch and parse RSS feeds, extract titles/links/dates/descriptions, and return a markdown summary table.  
- planning-with-files — Manus-style file-based planning for complex tasks (create and maintain task_plan.md, findings.md, progress.md; rules for when/how to write/read/update).  
- theme-factory — A theme toolkit (10 preset themes + ability to generate/apply a custom theme) for styling slides/docs/landing pages (color palettes, fonts, application process).  
- youtube-downloader — (listed) Skill to download YouTube videos with selectable quality/format and audio-only MP3 options.  
- web-search — Guidance and a bundled script for high-quality web searches, query planning, execution, result synthesis, and failure handling.

I loaded the instruction details for the listed skills (except a read error occurred when attempting to load the youtube-downloader instruction file, though it was present in the initial list). Would you like me to show more details from any of these skills or run a specific skill for you?

In [16]:
agent = ds.Agent('BA', tools=tools)
Markdown(agent.resp('What is the weather like in Beijing China today and tomorrow?'))

ValueError: wrapper loop when unwrapping <function get_current_weather at 0x0000019DC76747C0>

In [13]:
agent = ds.Agent('AAA', model='gpt-5-mini', mcp_servers=mcp_servers, tools=tools)
Markdown(await agent.resp('What is the weather like in Beijing China today and tomorrow?'))

[2026-01-21 16:23:18][ERROR][Failed to connect MCP Sevice 'http://localhost:8000/mcp', error: Client failed to connect: All connection attempts failed]


Assistant:

    Function Call:
        Name: get_current_weather
        Arguments: {"location":"Beijing, China","unit":"celsius"}
        Call ID: call_0WuC7omiYQ9dLc8tlJpDNIWu


    Function Call:
        Name: get_tomorrows_weather
        Arguments: {"location":"Beijing, China","unit":"celsius"}
        Call ID: call_JBJNwm6FI48XQ2R0ewShwXzn


--------------------------------------------------------------------------------------------------------

Assistant:

    I checked the weather, but the weather service returned implausible temperatures (it said 75 °C
    today and 60 °C tomorrow). I don’t trust those numbers.

    It did report conditions: “sunny today” and “rainy tomorrow.” How would you like me to proceed?
    - I can retry the forecast now (tell me whether you prefer Celsius or Fahrenheit).
    - Or I can look up a reliable forecast source and give a precise, sourced report.

-------------------------------------------------------------------------------------------------

I checked the weather, but the weather service returned implausible temperatures (it said 75 °C today and 60 °C tomorrow). I don’t trust those numbers.

It did report conditions: “sunny today” and “rainy tomorrow.” How would you like me to proceed?
- I can retry the forecast now (tell me whether you prefer Celsius or Fahrenheit).  
- Or I can look up a reliable forecast source and give a precise, sourced report.

In [7]:
agent = ds.Agent('AAA', model='gpt-5-mini', mcp_servers=mcp_servers, tools=tools)
Markdown(await agent.resp('What is the weather like in Beijing China today and tomorrow?'))

Assistant:

    Function Call:
        Name: get_current_weather
        Arguments: {"location":"Beijing, China","unit":"celsius"}
        Call ID: call_ELOuOuSfMFSK6tlflzwbhxuI


    Function Call:
        Name: get_tomorrows_weather
        Arguments: {"location":"Beijing, China","unit":"celsius"}
        Call ID: call_wPgM6MQjqtRW2iUkS7cYR82i


--------------------------------------------------------------------------------------------------------

Assistant:

    I checked the forecast but the weather source returned implausible values (it reported 75 °C
    today and 60 °C tomorrow). Those are almost certainly Fahrenheit values mis‑labelled as Celsius.
    Converting those numbers:

    - Today (now): sunny, about 75 °F → ~24 °C (rounded from 23.9 °C).
    - Tomorrow: rainy, about 60 °F → ~16 °C (rounded from 15.6 °C).

    Would you like me to:
    - Re-check with another source?
    - Provide more details (hourly forecast, precipitation chance, wind)?
    - Show the temperatures

I checked the forecast but the weather source returned implausible values (it reported 75 °C today and 60 °C tomorrow). Those are almost certainly Fahrenheit values mis‑labelled as Celsius. Converting those numbers:

- Today (now): sunny, about 75 °F → ~24 °C (rounded from 23.9 °C).  
- Tomorrow: rainy, about 60 °F → ~16 °C (rounded from 15.6 °C).

Would you like me to:
- Re-check with another source?  
- Provide more details (hourly forecast, precipitation chance, wind)?  
- Show the temperatures only in Celsius or only in Fahrenheit?

In [27]:
# https://www.ft.com/rss/home/international
agent = ds.Agent('LAA', model='gpt-5-nano', mcp_servers=mcp_servers, tools=tools)
Markdown(await agent.resp(r'parse the rss feed from "https://www.ft.com/rss/home/international", '
 'extract the all news from rss, merge them in a markdown table with the format provided in instructions and output translated in Chinese.'))

Assistant:

    Reasoning:
    Okay, let's tackle this user request step by step. The user wants to parse an RSS feed
    from "https://www.ft.com/rss/home/international", extract all the news items, merge them into a
    markdown table formatted according to the instructions provided, and then output the result in
    Chinese.

    First, I need to check which of the available tools can help with parsing the RSS feed. Looking
    at the list, there's a function called parse_rss_feed that takes a feed_url as an argument. That
    should handle the initial step of fetching the RSS data.

    Next, once the RSS feed is parsed, the user wants all news items extracted. The parse_rss_feed
    function should return the latest entries, which I can then process. But I need to make sure
    that the function actually returns all the necessary details like title, link, description, and
    publication date for each item.

    After extracting the data, the next challenge is formatting it into a

已按照要求将提取的国际新闻整理成Markdown表格，并保存为`international_news.md`。文件包含以下内容：

- **标题**：新闻标题（中文翻译）
- **描述**：新闻摘要（中文翻译）
- **链接**：原始英文文章链接
- **发布日期**：统一格式化为 `YYYY-MM-DD`

示例条目：
```
| 丹麦在格陵兰部署额外部队 | 股票下跌，黄金上涨，欧洲国家争相应对唐纳德·特朗普威胁的关税。 | [链接](https://www.ft.com/content/8d2b76e7-4450-4859-8128-866bc7cbfb90) | 2026-01-19 |
```

文件共包含13条新闻，涵盖国际政治、经济、社会事件等主题。

In [11]:
agent = ds.Agent('AAA', name='Tom', description='Websearch agent', model='gpt-5-mini', mcp_servers=mcp_servers, tools=tools)
Markdown(await agent.resp('Web search ther resent news about "贝森特称欧洲过于软弱，无法保障格陵兰安全". Return at least 5 relevant news. '))   

Tom:

    Function Call:
        Name: read_skill_instruction
        Arguments: {"skill_name":"web-search"}
        Call ID: call_NTf4pPr3o3dsG2nEJiKZ5IUz


--------------------------------------------------------------------------------------------------------

Tom:

    Function Call:
        Name: execute_skill_script
        Arguments: {"skill_name":"web-search","script_link":"scripts\\web_search.py"}
        Call ID: call_bCYBhzcMoeykSttb7gnSt3gv


--------------------------------------------------------------------------------------------------------

Tom:

    Function Call:
        Name: execute_skill_script
        Arguments: {"skill_name":"web-
    search","script_link":"scripts\\web_search.py","parameters":{}}
        Call ID: call_IuDsukqqCmLnZzBymVZ5Lweo


--------------------------------------------------------------------------------------------------------

Tom:

    Function Call:
        Name: execute_skill_script
        Arguments: {"skill_name":"web-
    search","s

I searched for recent news about “贝森特称欧洲过于软弱，无法保障格陵兰安全” and found multiple relevant reports. Below are 6 items (title — source — URL — short note/date when available). Tell me which ones you want summarized or if you want more results.

1) 贝森特 欧洲八国为夺取格陵兰岛特朗变疯狂 — RFI（法广 中文）  
   https://www.rfi.fr/cn/中国/20260117-威斯特欧洲八国-为夺取格陵兰岛特朗变疯狂  
   Note: RFI piece (2026-01-17 in URL) discussing competing powers and security debates over Greenland; mentions European responses and US positions.

2) 特朗普可以如何“拿下”格陵兰？ — BBC 中文  
   https://www.bbc.com/zhongwen/articles/c5y403gj0ggo/simp  
   Note: BBC Chinese background on strategic interest in Greenland and foreign deployments; covers US and European roles.

3) 格陵兰计划受阻 特朗普对欧洲8国加征关税 — 德国之声 (DW 中文)  
   https://www.dw.com/zh/格陵兰计划受阻-特朗普对欧洲8国加征关税/a-75552165  
   Note: DW report on diplomatic tensions around Greenland, including US pressure and European reactions.

4) 欧派1+2+13… 白宫：已阅，不影响美国的格陵兰岛计划 — 观察者网（Guancha）  
   https://www.guancha.cn/internation/2026_01_16_804021.shtml  
   Note: Coverage of European deployments/commitments and comments questioning whether Europe can ensure Greenland’s security.

5) 特朗普宣布向欧8国加征关税，直至“完全收购格陵兰岛” — 凤凰网（Ifeng）  
   https://news.ifeng.com/c/8q00jeWUQBa  
   Note: Reports on US diplomatic/financial pressure and mentions debate over Europe’s capacity to back Greenland security.

6) 美总统将对八个欧洲国家加征关税 — 联合早报 / yzaobao  
   https://www.yzaobao.com/news/politics/202601/1862599.html  
   Note: Lianhe Zaobao summary of US actions and implications for Greenland security; references European responses and statements questioning Europe’s role.

Would you like me to:
- Open and extract key paragraphs from any of these articles?  
- Provide a Chinese-language summary of each?  
- Search for additional or English-language coverage (e.g., AP, NYT, Reuters)?

In [35]:

agent = ds.Agent('AAA', name='Tom', description='Websearch agent', model='gpt-5-mini', mcp_servers=mcp_servers, tools=tools)
Markdown(await agent.resp('Web search ther resent news from Bosch of the current week. Return at least 10 news. ' 
' write the results to a file called "Bosch_news.md"'))   

Tom:

    Function Call:
        Name: read_skill_instruction
        Arguments: {"skill_name":"web-search"}
        Call ID: call_96wvzuXUxS0oF9ApMjLT7yLn


--------------------------------------------------------------------------------------------------------

Tom:

    Function Call:
        Name: execute_skill_script
        Arguments: {"skill_name":"web-search","script_link":"scripts/web_search.py"}
        Call ID: call_fmqtzJmXVIIrKtCyzvOFBEJX


--------------------------------------------------------------------------------------------------------

Tom:

    Function Call:
        Name: read_skill_reference
        Arguments: {"skill_name":"web-search","reference_link":"scripts/web_search.py"}
        Call ID: call_u8IumwYJMb6AeD6ZJDKQ2znu


--------------------------------------------------------------------------------------------------------

Tom:

    Function Call:
        Name: execute_skill_script
        Arguments: {"skill_name":"web-
    search","script_link":"scripts

I searched recent news about Bosch and wrote the results to Bosch_news.md. The file contains at least 10 news items (CES coverage, eBike Flow theft-protection updates, robotics partnership, smart crib, Bosch Tech Compass, and related press releases and coverage) with links, dates, and short summaries.

If you want:
- Only items strictly from this week (exact dates Jan 12–18, 2026) — I can filter and update the file.
- More than 10 items or full-text excerpts for any item — I can add those.
- The file in a different format (DOCX, PDF) — I can convert it.

In [36]:
agent.history.to_dataframe()

,msg_id,name,role,content,context_level,function_name,call_id,usage,call_output,session_id
0,0,User,user,Web search ther resent news from Bosch of the ...,0.0,NaN,NaN,NaN,NaN,0
1,1,Tom,None,NaN,0.0,read_skill_instruction,call_96wvzuXUxS0oF9ApMjLT7yLn,"{'input_tokens': 1450, 'input_tokens_details':...",NaN,0
2,2,System,system,NaN,0.0,NaN,call_96wvzuXUxS0oF9ApMjLT7yLn,NaN,"""---\nname: web-search \ndescription: Guide...",0
3,3,Tom,None,NaN,0.0,execute_skill_script,call_fmqtzJmXVIIrKtCyzvOFBEJX,"{'input_tokens': 2403, 'input_tokens_details':...",NaN,0
4,4,System,system,NaN,0.0,NaN,call_fmqtzJmXVIIrKtCyzvOFBEJX,NaN,"""Error executing tool execute_skill_script: Ag...",0
5,5,Tom,None,NaN,0.0,read_skill_reference,call_u8IumwYJMb6AeD6ZJDKQ2znu,"{'input_tokens': 2470, 'input_tokens_details':...",NaN,0
6,6,System,system,NaN,0.0,NaN,call_u8IumwYJMb6AeD6ZJDKQ2znu,NaN,"""import argparse\nimport json\nfrom datasurfer...",0
7,7,Tom,None,NaN,0.0,execute_skill_script,call_26tUL8QSLJI4d93bGZZutC2o,"{'input_tokens': 2664, 'input_tokens_details':...",NaN,0
8,8,System,system,NaN,0.0,NaN,call_26tUL8QSLJI4d93bGZZutC2o,NaN,"""[\n {\n \""title\"": \""Home | Bosch i...",0
9,9,Tom,None,NaN,NaN,execute_skill_script,call_PZDfS2bLJeQFBNigmqJpoowY,NaN,NaN,0
